Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import joblib

Загрузка файлов

In [2]:
model = joblib.load("../models/model_oned.pkl")

test_data = pd.read_excel("../data/test.xlsx")
test_data['dt'] = pd.to_datetime(test_data['dt'])

Создание лагов

In [3]:
lags = 6
for i in range(1, lags + 1):
    test_data[f'lag_{i}'] = test_data['Цена на арматуру'].shift(i)

test_data.fillna(method='bfill', inplace=True)

C:\Users\Евгений\AppData\Local\Temp\ipykernel_22288\3815960519.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_data.fillna(method='bfill', inplace=True)


Функция прогнозирования на 6 недель вперед

In [4]:
def predict_n_weeks_ahead(model, features, n_weeks=6):
    predictions = []
    
    current_features = features.copy()

    for _ in range(n_weeks):
        # Делаем предсказание
        prediction = model.predict([current_features])[0]
        predictions.append(prediction)

        # Обновляем лаги
        current_features = np.roll(current_features, shift=-1)
        current_features[-1] = prediction
    
    return predictions

Функция определения длительности падения

In [5]:
def determine_n_weeks(predictions):
    n_weeks = 1
    for i in range(1, len(predictions)):
        if predictions[i] < predictions[i - 1]:  # Если цена падает
            n_weeks += 1
        elif predictions[i] == predictions[i - 1]:  # Если цена не меняется
            continue
        else:  # Если цена растет
            break 
    return min(n_weeks, 6)

Добавляем колонку с рекомендацией по объему тендера

In [6]:
test_data['N_weeks'] = test_data.apply(
    lambda row: determine_n_weeks(predict_n_weeks_ahead(model, row.iloc[2:].values)), axis=1
)

Приводим таблицу к готовому виду

In [7]:
# Переименовываем колонку N_weeks
test_data.rename(columns={'N_weeks': 'Объем'}, inplace=True)

# Сохраняем только нужные колонки
columns_to_keep = ['dt', 'Цена на арматуру', 'Объем']  
test_data = test_data[columns_to_keep]

Сохраняем результат

In [8]:
test_data.to_excel("../data/test_with_recommendations.xlsx", index=False)